## En este notebook se va a llevar a cabo todo lo relacionado al modelo de Machine learning

Importamos las bibliotecas necesarias

In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split

In [ ]:
#df_ML= pd.read_csv('df_ML.csv')

Eliminamos columnas que no aportan información ni valor para este proceso:

- Unnamed: 0 -> esta columna se agregó automaticamente cuando creamos el archivo .csv
- developer
- release_date
- playtime_forever

In [ ]:
#df_ML=df_ML.drop(columns=['Unnamed: 0','developer', 'release_date', 'playtime_forever'])

- Se configura el modelo de Machine Learning basándonos en la similaridad del coseno
- El problema con este modelo, es que necesita mucha memoria de procesamiento

In [ ]:
# Paso 1: Preprocesamiento y vectorización
#textos = df_ML['genres']
#vectorizer = TfidfVectorizer()
#matriz_tfidf = vectorizer.fit_transform(textos)

# Paso 2: Reducción de dimensionalidad con PCA
#ipca = IncrementalPCA(n_components=20)  # Por ejemplo, reduce a 50 componentes
#matriz_ipca = ipca.fit_transform(matriz_tfidf)


# Paso 3: Cálculo de similitud del coseno después de PCA
#similitud_pca = cosine_similarity(matriz_ipca)

In [ ]:
#similitud = similitud_pca

In [ ]:
#Filtramos el dataframe solo con los datos que tengan valor 2 en sentiment_analysis y que sea True en recommend
#ya que demanda mucha memoria RAM
#df_ML=df_ML[df_ML['sentiment_analysis']==2]
#df_ML=df_ML[df_ML['recommend']== True]
#Igualmente haciendo ese filtrado de datos, sigue sin poder funcionar.

In [ ]:
#def obtener_recomendaciones(titulo, n=5):
#    indice_titulo = df_ML[df_ML['app_name'] == titulo].index[0]
#    similitudes_titulo = similitud[indice_titulo]
#    indices_recomendaciones = similitudes_titulo.argsort()[-n-1:-1][::-1]
#    recomendaciones = df_ML.loc[indices_recomendaciones, 'app_name']
    
    # Convierte la serie en un diccionario
#    recomendaciones_dict = recomendaciones.to_dict()

#    return recomendaciones_dict

## Modelo de machine learning FUNCIONAL <br>
Importamos la librería pickle ya que vamos a guardar el modelo de entrenamiento en un archivo .pkl (el cual es un archivo de formato binario que se utiliza para modelos de entrenamiento)para asi luego crear la función de recomendación.

In [1]:
import pickle

In [3]:
# Crear una instancia del codificador
label_encoder = LabelEncoder()

# Cargar los datos
df_ml = pd.read_csv("df_ML.csv")
# Crear una nueva columna llamada genres_encoded, que tiene los generos codificados como int.
df_ml["genres_encoded"] = label_encoder.fit_transform(df_ml["genres"])

# Crear un diccionario de los títulos asociados a cada item_id
titles_by_item_id = {}
for i in range(len(df_ml)):
    titles_by_item_id[df_ml.loc[i, "item_id"]] = df_ml.loc[i, "app_name"]

# Crear el modelo K-Nearest Neighbors
k = 5
model = KNeighborsClassifier(n_neighbors=k)

# Entrenar el modelo
model.fit(df_ml[['genres_encoded']], df_ml['app_name'])

# Guardar el modelo
with open('modelo.pkl', 'wb') as f:
    pickle.dump(model, f)

# Guardar el diccionario
with open('titles_by_item_id.pkl', 'wb') as f:
    pickle.dump(titles_by_item_id, f)

In [4]:
def get_recommendations(item_id: int):

    # Cargar el modelo 
    with open("modelo.pkl", "rb") as f:
        model = pickle.load(f)

    # Cargar el diccionario de títulos
    with open("titles_by_item_id.pkl", "rb") as f:
        titles_by_item_id = pickle.load(f)

    # Buscar el género codificado del juego proporcionado por el usuario
    input_game = df_ml[df_ml["item_id"] == item_id]["genres_encoded"].values[0]

    # Encontrar los juegos más similares
    _, indices = model.kneighbors([[input_game]])

    # Obtener los títulos de los juegos similares
    similar_games = [titles_by_item_id[df_ml.loc[i, "item_id"]] for i in indices[0]]

    # Devolver un diccionario de los títulos
    return {"similar_games": similar_games}

In [5]:
get_recommendations(70)

c:\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


{'similar_games': ['Half-Life',
  'Half-Life',
  "Tom Clancy's Rainbow Six® Siege",
  'Half-Life',
  'Half-Life']}